## Import libraries

In [1]:
import numpy as np
import pandas as pd
import shap
import lime
import os
import pickle


from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn import svm

from sklearn import tree
import xgboost as xgb
from sklearn.datasets import make_regression
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


import matplotlib.pyplot as plt

## Dataset Importing

In [2]:
df = pd.read_csv('/Users/saumenduroy/Documents/PhD Life/Explainability + SOAR/Explainers code/JM1_Single dataset and single file/Data/jm1.csv')
obj_list = df.columns[df.dtypes == 'object'].values.tolist()
for l in obj_list:
    df[l] = pd.to_numeric(df[l], errors='coerce')
df["defects"] = df["defects"].astype(int)
df.dropna(inplace=True)

## Select a specific row or column from the dataset and dataset splitting

In [3]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Dataset Balancing

In [1158]:
## from imblearn.over_sampling import RandomOverSampler

In [1159]:
## from collections import Counter
## oversample =  RandomOverSampler()
##X_train,y_train = oversample.fit_resample(X_train,y_train)

In [1160]:
#Counter(y_train)

In [1161]:
#Counter(y_test)

In [4]:
print("X_Train shape: ", X_train.shape)
print("Train label shape: ", y_train.shape)
print("X_Test shape: ", X_test.shape)
print("Test label shape: ", y_test.shape)

X_Train shape:  (7616, 21)
Train label shape:  (7616,)
X_Test shape:  (3264, 21)
Test label shape:  (3264,)


In [5]:
X_train

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount
4029,4.0,1.0,1.0,1.0,5.0,11.61,0.67,1.50,7.74,17.41,...,0.97,2,0,0,0,3.0,2.0,3.0,2.0,1.0
3671,7.0,1.0,1.0,1.0,23.0,89.86,0.30,3.33,26.96,299.53,...,16.64,4,0,1,0,6.0,9.0,13.0,10.0,1.0
6238,30.0,3.0,3.0,2.0,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0,0,0,0,0.0,0.0,0.0,0.0,5.0
1432,104.0,13.0,10.0,13.0,265.0,1640.30,0.04,27.14,60.44,44517.84,...,2473.21,74,25,3,0,23.0,50.0,147.0,118.0,25.0
4140,17.0,3.0,1.0,3.0,37.0,162.52,0.09,10.67,15.24,1733.50,...,96.31,12,0,3,0,12.0,9.0,21.0,16.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5738,17.0,2.0,1.0,2.0,45.0,211.52,0.14,7.29,29.03,1541.07,...,85.62,10,1,3,0,12.0,14.0,28.0,17.0,3.0
5195,14.0,3.0,3.0,2.0,31.0,133.98,0.09,11.14,12.02,1492.92,...,82.94,10,0,2,0,13.0,7.0,19.0,12.0,5.0
5394,20.0,2.0,1.0,2.0,85.0,384.50,0.05,18.67,20.60,7177.38,...,398.74,14,3,1,0,14.0,9.0,61.0,24.0,3.0
862,33.0,4.0,3.0,2.0,90.0,454.00,0.04,24.43,18.58,11090.46,...,616.14,18,5,8,0,19.0,14.0,54.0,36.0,7.0


In [6]:
df.columns

Index(['loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e', 'b', 't',
       'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment', 'uniq_Op',
       'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount', 'defects'],
      dtype='object')

## Checking the null value

In [7]:
df.isnull().sum()

loc                  0
v(g)                 0
ev(g)                0
iv(g)                0
n                    0
v                    0
l                    0
d                    0
i                    0
e                    0
b                    0
t                    0
lOCode               0
lOComment            0
lOBlank              0
locCodeAndComment    0
uniq_Op              0
uniq_Opnd            0
total_Op             0
total_Opnd           0
branchCount          0
defects              0
dtype: int64

In [8]:
df.columns[df.dtypes == 'object']

Index([], dtype='object')

In [9]:
df.dtypes

loc                  float64
v(g)                 float64
ev(g)                float64
iv(g)                float64
n                    float64
v                    float64
l                    float64
d                    float64
i                    float64
e                    float64
b                    float64
t                    float64
lOCode                 int64
lOComment              int64
lOBlank                int64
locCodeAndComment      int64
uniq_Op              float64
uniq_Opnd            float64
total_Op             float64
total_Opnd           float64
branchCount          float64
defects                int64
dtype: object

In [1000]:
#our_rf_model = RandomForestClassifier(random_state=0)
#our_rf_model.fit(X_train, y_train) 

In [10]:
## For SVM data fitting
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

## Create classifier object, Train, and Test 

In [11]:
# Create classifer object
our_rf_model = RandomForestClassifier()
our_dt_model = DecisionTreeClassifier()
our_lr_model = LogisticRegression()
our_mlp_model = MLPClassifier()
our_xgb_model = XGBClassifier()
our_svm_model = svm.SVC()

# Train Classifer
our_rf_model = our_rf_model.fit(X_train,y_train)
our_dt_model = our_dt_model.fit(X_train,y_train)
our_lr_model = our_lr_model.fit(X_train,y_train)
our_mlp_model = our_mlp_model.fit(X_train,y_train)
our_xgb_model = our_xgb_model.fit(X_train,y_train)
our_svm_model = our_svm_model.fit(X_train [:500], y_train[:500])

#Predict the response for test dataset

y_pred_our_rf_model = our_rf_model.predict(X_test)
y_pred_our_dt_model = our_dt_model.predict(X_test)
y_pred_our_lr_model = our_lr_model.predict(X_test)
y_pred_our_mlp_model = our_mlp_model.predict(X_test)
y_pred_our_xgb_model = our_xgb_model.predict(X_test)
y_pred_our_svm_model = our_svm_model.predict(X_test)

## Measuring Accuracy

In [12]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy for Random Forest (RF):",metrics.accuracy_score(y_test, y_pred_our_rf_model))
print("Accuracy for Decission Tree (DT):",metrics.accuracy_score(y_test, y_pred_our_dt_model))
print("Accuracy for Logistic Regression (LR):",metrics.accuracy_score(y_test, y_pred_our_lr_model))
print("Accuracy for Multi-Layer Perceptron Neural Network (MLP):",metrics.accuracy_score(y_test, y_pred_our_mlp_model))
print("Accuracy for Gradient Boosting (XGB):",metrics.accuracy_score(y_test, y_pred_our_xgb_model))
print("Accuracy for Support Vector Machine (SVM):",metrics.accuracy_score(y_test, y_pred_our_svm_model))


Accuracy for Random Forest (RF): 0.8146446078431373
Accuracy for Decission Tree (DT): 0.7377450980392157
Accuracy for Logistic Regression (LR): 0.7999387254901961
Accuracy for Multi-Layer Perceptron Neural Network (MLP): 0.4990808823529412
Accuracy for Gradient Boosting (XGB): 0.8063725490196079
Accuracy for Support Vector Machine (SVM): 0.8017769607843137


## Precision, Recall, and f1 Score, and AUC for RF

In [13]:
print("Precision:",metrics.precision_score(y_test, y_pred_our_rf_model, average = "macro"))
print("Recall:",metrics.recall_score(y_test, y_pred_our_rf_model, average = "macro"))
print("f1 score:",metrics.f1_score(y_test,y_pred_our_rf_model, average = "macro"))

Precision: 0.7044475296821141
Recall: 0.6011215003063481
f1 score: 0.6189840384735976


In [14]:
from sklearn import metrics
u_value= list(y_test.unique())
all_auc = []
for i in u_value:
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_our_rf_model, pos_label=i)
    all_auc.append(metrics.auc(fpr, tpr))
print(all_auc)
print("AUC for RF:",sum(all_auc)/len(all_auc))

[0.39887849969365197, 0.601121500306348]
AUC for RF: 0.5


In [15]:
list(y_test.unique())

[0, 1]

## Precision, Recall, and f1 Score, and AUC for DT

In [16]:
print("Precision:",metrics.precision_score(y_test, y_pred_our_dt_model, average = "macro"))
print("Recall:",metrics.recall_score(y_test, y_pred_our_dt_model, average = "macro"))
print("f1 score:",metrics.f1_score(y_test,y_pred_our_dt_model, average = "macro"))

Precision: 0.5917276800537694
Recall: 0.5946864150172709
f1 score: 0.5931158417773688


In [17]:
from sklearn import metrics
u_value= list(y_test.unique())
all_auc = []
for i in u_value:
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_our_dt_model, pos_label=i)
    all_auc.append(metrics.auc(fpr, tpr))
print(all_auc)
print("AUC for DT:",sum(all_auc)/len(all_auc))

[0.4053135849827291, 0.5946864150172709]
AUC for DT: 0.5


## Precision, Recall, and f1 Score, and AUC for LR

In [18]:
print("Precision:",metrics.precision_score(y_test, y_pred_our_lr_model, average = "micro"))
print("Recall:",metrics.recall_score(y_test, y_pred_our_lr_model, average = "macro"))
print("f1 score:",metrics.f1_score(y_test,y_pred_our_lr_model, average = "macro"))

Precision: 0.7999387254901961
Recall: 0.5171696990685243
f1 score: 0.4884438616993165


In [19]:
from sklearn import metrics
u_value= list(y_test.unique())
all_auc = []
for i in u_value:
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_our_lr_model, pos_label=i)
    all_auc.append(metrics.auc(fpr, tpr))
print(all_auc)
print("AUC for LR:",sum(all_auc)/len(all_auc))

[0.4828303009314757, 0.5171696990685243]
AUC for LR: 0.5


## Precision, Recall, and f1 Score, and AUC for MLP

In [20]:
print("Precision:",metrics.precision_score(y_test, y_pred_our_mlp_model, average = "macro"))
print("Recall:",metrics.recall_score(y_test, y_pred_our_mlp_model, average = "macro"))
print("f1 score:",metrics.f1_score(y_test,y_pred_our_mlp_model, average = "macro"))

Precision: 0.550490557122806
Recall: 0.5774288073736648
f1 score: 0.473683743364018


In [21]:
from sklearn import metrics
u_value= list(y_test.unique())
all_auc = []
for i in u_value:
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_our_mlp_model, pos_label=i)
    all_auc.append(metrics.auc(fpr, tpr))
print(all_auc)
print("AUC for MLP:",sum(all_auc)/len(all_auc))

[0.4225711926263353, 0.5774288073736648]
AUC for MLP: 0.5


## Precision, Recall, and f1 Score, and AUC for XGB

In [22]:
print("Precision:",metrics.precision_score(y_test, y_pred_our_xgb_model, average = "macro"))
print("Recall:",metrics.recall_score(y_test, y_pred_our_xgb_model, average = "macro"))
print("f1 score:",metrics.f1_score(y_test,y_pred_our_xgb_model, average = "macro"))

Precision: 0.6785333990770032
Recall: 0.5953825798946873
f1 score: 0.6107172916155574


In [23]:
from sklearn import metrics
u_value= list(y_test.unique())
all_auc = []
for i in u_value:
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_our_xgb_model, pos_label=i)
    all_auc.append(metrics.auc(fpr, tpr))
print(all_auc)
print("AUC for XGB:",sum(all_auc)/len(all_auc))

[0.40461742010531265, 0.5953825798946873]
AUC for XGB: 0.5


## Precision, Recall, and f1 Score, and AUC for SVM

In [24]:
print("Precision:",metrics.precision_score(y_test, y_pred_our_svm_model, average = "micro"))
print("Recall:",metrics.recall_score(y_test, y_pred_our_svm_model, average = "micro"))
print("f1 score:",metrics.f1_score(y_test,y_pred_our_svm_model, average = "micro"))

Precision: 0.8017769607843137
Recall: 0.8017769607843137
f1 score: 0.8017769607843137


In [25]:
from sklearn import metrics
u_value= list(y_test.unique())
all_auc = []
for i in u_value:
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_our_svm_model, pos_label=i)
    all_auc.append(metrics.auc(fpr, tpr))
print(all_auc)
print("AUC for SVM:",sum(all_auc)/len(all_auc))

[0.49979754388769015, 0.5002024561123098]
AUC for SVM: 0.5


## CV Score for RF

In [26]:
scores = cross_val_score(our_rf_model, X_train, y_train, cv=10)
print('Cross-Validation Accuracy Scores for RF', scores)

Cross-Validation Accuracy Scores for RF [0.81233596 0.82677165 0.82939633 0.80183727 0.83595801 0.81496063
 0.8173456  0.81865966 0.80551905 0.81997372]


## After CV: Minimum, Mean, and Maximum Value for RF

In [27]:
scores = pd.Series(scores)
scores.min(), scores.mean(), scores.max()

(0.8018372703412073, 0.8182757871429016, 0.8359580052493438)

In [28]:
print('Mean Accuracy Scores for RF', scores.mean())

Mean Accuracy Scores for RF 0.8182757871429016


## CV Score for DT

In [29]:
scores = cross_val_score(our_dt_model, X_train, y_train, cv=10)
print('Cross-Validation Accuracy Scores for DT', scores)

Cross-Validation Accuracy Scores for DT [0.72834646 0.76509186 0.75984252 0.74015748 0.76771654 0.76115486
 0.74244415 0.74375821 0.75558476 0.75164258]


## After CV: Minimum, Mean, and Maximum Value for DT

In [30]:
scores = pd.Series(scores)
scores.min(), scores.mean(), scores.max()

(0.7283464566929134, 0.7515739409052187, 0.7677165354330708)

In [31]:
print('Mean Accuracy Scores for DT', scores.mean())

Mean Accuracy Scores for DT 0.7515739409052187


## CV Score for LR

In [32]:
scores = cross_val_score(our_lr_model, X_train, y_train, cv=10)
print('Cross-Validation Accuracy Scores for LR', scores)

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the doc

Cross-Validation Accuracy Scores for LR [0.80708661 0.80839895 0.81102362 0.79790026 0.81364829 0.82152231
 0.8173456  0.80551905 0.75558476 0.80026281]


lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the doc

## After CV: Minimum, Mean, and Maximum Value for LR

In [33]:
scores = pd.Series(scores)
scores.min(), scores.mean(), scores.max()

(0.7555847568988173, 0.8038292273255593, 0.821522309711286)

In [34]:
print('Mean Accuracy Scores for LR', scores.mean())

Mean Accuracy Scores for LR 0.8038292273255593


## CV Score for MLP

In [35]:
scores = cross_val_score(our_mlp_model, X_train, y_train, cv=10)
print('Cross-Validation Accuracy Scores for MLP', scores)

Cross-Validation Accuracy Scores for MLP [0.7808399  0.67979003 0.81102362 0.80971129 0.71784777 0.79002625
 0.80683311 0.80551905 0.80289093 0.79894875]


## After CV: Minimum, Mean, and Maximum Value for MLP 

In [36]:
scores = pd.Series(scores)
scores.min(), scores.mean(), scores.max()

(0.6797900262467191, 0.7803430697969587, 0.8110236220472441)

In [37]:
print('Mean Accuracy Scores for MLP', scores.mean())

Mean Accuracy Scores for MLP 0.7803430697969587


## CV Score for XGB

In [38]:
scores = cross_val_score(our_xgb_model, X_train, y_train, cv=10)
print('Cross-Validation Accuracy Scores for XGB', scores)

Cross-Validation Accuracy Scores for XGB [0.80839895 0.80577428 0.82152231 0.79790026 0.80577428 0.81889764
 0.80683311 0.81471748 0.79632063 0.80026281]


## After CV: Minimum, Mean, and Maximum Value for XGB

In [39]:
scores = pd.Series(scores)
scores.min(), scores.mean(), scores.max()

(0.7963206307490145, 0.8076401750701004, 0.821522309711286)

In [40]:
print('Mean Accuracy Scores for XGB', scores.mean())

Mean Accuracy Scores for XGB 0.8076401750701004


## CV Score for SVM

In [41]:
scores = cross_val_score(our_svm_model, X_train, y_train, cv=10)
print('Cross-Validation Accuracy Scores for SVM', scores)

Cross-Validation Accuracy Scores for SVM [0.80971129 0.81233596 0.81233596 0.81102362 0.81496063 0.81102362
 0.81340342 0.80946124 0.8107753  0.80946124]


## After CV: Minimum, Mean, and Maximum Value for SVM

In [42]:
scores = pd.Series(scores)
scores.min(), scores.mean(), scores.max()

(0.80946123521682, 0.8114492258769888, 0.8149606299212598)

In [43]:
print('Mean Accuracy Scores for XGB', scores.mean())

Mean Accuracy Scores for XGB 0.8114492258769888
